In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [55]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        #fc - full connection
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, img):
        x = F.max_pool2d(F.relu(self.conv1(img)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        #flatter
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)
        

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


the learnable parameters are in `net.parameters`

In [56]:
params = list(net.parameters())

In [57]:
len(params)

10

In [58]:
params[1].size()

torch.Size([6])

In [59]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.1199, -0.0463, -0.0270, -0.0434, -0.1458,  0.0611,  0.0044, -0.0589,
         -0.0635,  0.0332]], grad_fn=<AddmmBackward>)


In [60]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [61]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.5156, grad_fn=<MseLossBackward>)


In [62]:
target.shape

torch.Size([1, 10])

In [63]:
print(loss.grad_fn)

If we run `backward` on loss, every parameter is set with diff w.r.t loss, which is what we want basically

In [64]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [65]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([0.0026, 0.0076, 0.0032, 0.0043, 0.0021, 0.0115])


In [79]:
for p in net.conv1.parameters():
    print(p.grad.size())

torch.Size([6, 1, 3, 3])
torch.Size([6])


`grad` is set for cov2d too

After getting `grad` we can change the weights easily, by reading the weights stored in `data` of each parameter

In [80]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

This weight updation can be done using differnet rules, this is absracted using `optim`

In [84]:
import torch.optim as optim

In [85]:
optimizer = optim.SGD(net.parameters(), lr=0.01)
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()